In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
hotel_df = pd.read_csv("H1.csv")

In [11]:
hotel_df["Date(Year and Week Number)"] = hotel_df['ArrivalDateYear'].astype(str) + hotel_df['ArrivalDateWeekNumber'].astype(str)
hotel_df

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,Date(Year and Week Number)
0,0,342,2015,July,27,1,0,0,2,0,...,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,201527
1,0,737,2015,July,27,1,0,0,2,0,...,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01,201527
2,0,7,2015,July,27,1,0,1,1,0,...,NULL,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,201527
3,0,13,2015,July,27,1,0,1,1,0,...,304,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02,201527
4,0,14,2015,July,27,1,0,2,2,0,...,240,NULL,0,Transient,98.00,0,1,Check-Out,2015-07-03,201527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40055,0,212,2017,August,35,31,2,8,2,1,...,143,NULL,0,Transient,89.75,0,0,Check-Out,2017-09-10,201735
40056,0,169,2017,August,35,30,2,9,2,0,...,250,NULL,0,Transient-Party,202.27,0,1,Check-Out,2017-09-10,201735
40057,0,204,2017,August,35,29,4,10,2,0,...,250,NULL,0,Transient,153.57,0,3,Check-Out,2017-09-12,201735
40058,0,211,2017,August,35,31,4,10,2,0,...,40,NULL,0,Contract,112.80,0,1,Check-Out,2017-09-14,201735


In [13]:
hotel_df.groupby("Date(Year and Week Number)")["IsCanceled"].sum().to_frame()

,IsCanceled
Date(Year and Week Number),
201527,41
201528,48
201529,87
201530,74
201531,101
...,...
20175,51
20176,91
20177,96
